In [1]:
import pandas as pd
import numpy as np
import os
import json
import statistics

In [2]:
d = "./result"

dirs_to_analyze = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o)) and "\\experiment" in os.path.join(d, o)]


In [3]:
rowsList = []
for curdir in dirs_to_analyze:
    row = {}
    try:
        config = json.load(open(curdir + "/config.json"))
        row['Executor Memory'] = int(config['jobClassParameters'][0]['systemParameters']['executorMemory'][:1])
        row['Batch Size'] = int(config['jobClassParameters'][0]['hyperParameters']['batchSize'])
        row['Max Epochs'] = int(config['jobClassParameters'][0]['hyperParameters']['maxEpoch'])
        row['Learning Rate'] = float(config['jobClassParameters'][0]['hyperParameters']['learningRate'])
        row['Learning Rate Decay'] = float(config['jobClassParameters'][0]['hyperParameters']['learningrateDecay'])
        row['Nodes'] = int(config['numberOfNodes'])
    except Exception as e:
        print("Error parsing config for folder " + curdir)
        print(e)
        
    try:
        accuracy = pd.read_json(curdir + "/accuracy.json").transpose()
        row['Top 1 mean'] = accuracy["top1"].mean()
        row['Top 1 std'] = accuracy["top1"].std()
        row['Top 5 mean'] = accuracy["top5"].mean()
        row['Top 5 std'] = accuracy["top5"].std()
        row['loss mean'] = accuracy["loss"].mean()
        row['loss std'] = accuracy["loss"].std()
        row['lossnum'] = accuracy["lossnum"].mean()
    except Exception as e:
        print("Error parsing accuracy for folder " + curdir)
        print(e)
        
        
    try:
        time_summary = json.load(open(curdir + "/summary.json"))
        service_times = []
        for job in time_summary['jobs']:
            service_time = float(job['end']) - float(job['arrive'])
            service_times.append(service_time)
        row['Service Time mean'] = statistics.mean(service_times)
        row['Service Time std'] = statistics.stdev(service_times)
    except Exception as e:
        print("Error parsing time summary for folder " + curdir)
        print(e)
        
    
    rowsList.append(row) 
    

analysis = pd.DataFrame(rowsList)
print(analysis)

Error parsing accuracy for folder ./result\experiment1_0.01_32_10_2G_2_0.01
Expected object or value
Error parsing time summary for folder ./result\experiment1_0.01_32_10_2G_2_0.01
[Errno 2] No such file or directory: './result\\experiment1_0.01_32_10_2G_2_0.01/summary.json'
    Executor Memory  Batch Size  Max Epochs  Learning Rate  \
0                 1         128          10          0.001   
1                 1         128          10          0.001   
2                 2         128          10          0.001   
3                 1         128           5          0.001   
4                 1         128           5          0.001   
5                 2         128           5          0.001   
6                 2         128           5          0.001   
7                 1          32          10          0.001   
8                 1          32          10          0.001   
9                 2          32          10          0.001   
10                2          32          1